In [4]:
import pinecone

# Initialize Pinecone
PINECONE_API_KEY = '923773a3-bc74-4a9b-85a4-c5dff73509de'
PINECONE_API_ENV = 'us-east-1'

# Create a Pinecone client instance
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)

In [5]:
index_name = "emergency-index"
pc.create_index(
    name=index_name,
    dimension=1024,  # Dimension for the multilingual-e5-large model
    metric="cosine",
    spec=pinecone.ServerlessSpec(
        cloud="aws",
        region="us-east-1"  # Choose the region according to your requirements
    )
)

In [6]:
import json
with open('emergencies.json') as json_file:
    data = json.load(json_file)


In [9]:
for emergency in data['emergencies']:
    emergency_text = emergency['emergency']
    print(emergency_text)
    next_step = emergency['next_step']
    print(next_step)

Patient is not breathing
Start CPR immediately. Position your hands on the center of the patient's chest and push hard and fast at a rate of 100-120 compressions per minute. Ensure the airway is clear and give rescue breaths every 30 compressions.
Severe bleeding
Apply direct pressure to the wound using a clean cloth or bandage. Elevate the injured area above the heart if possible. Do not remove the cloth even if it becomes soaked with blood; instead, add more layers.
Chest pain
Have the patient sit down and remain calm. Loosen any tight clothing. If the patient has prescribed nitroglycerin, help them take it. Monitor the patientâ€™s condition and be prepared to start CPR if necessary.
Burns
Cool the burn with running water for at least 10 minutes. Do not apply ice, creams, or oily substances. Cover the burn with a sterile, non-stick bandage or clean cloth. Avoid bursting any blisters.
Fracture
Keep the injured limb in the position you found it and immobilize it with a splint or by sec

In [16]:
# Prepare the data for upsert
upsert_data = []
for emergency in data['emergencies']:
    emergency_text = emergency['emergency']
    next_step = emergency['next_step']
    
    # Embed using Pinecone's built-in embedding with the required input_type parameter
    embedding_response = pc.inference.embed(
        model="multilingual-e5-large",
        inputs=[emergency_text],
        parameters={"input_type": "passage"}  # Specify the input type
    )
    
    # Access the embedding directly from the response
    vector = embedding_response[0]['values']  # Get the first embedding's values

    # Append the data in the required format for upsert
    upsert_data.append({
        "id": emergency_text,  # Using the emergency text as the ID
        "values": vector,  # Use the embedding values directly
        "metadata": {"next_step": next_step}  # Metadata for the embedding
    })

# Upsert data into Pinecone
index = pc.Index(index_name)
index.upsert(vectors=upsert_data)




{'upserted_count': 52}

In [26]:
def query_emergency(emergency_query):
    # Embed the query text using Pinecone's built-in embedding with the required input_type parameter
    query_vector = pc.inference.embed(
        model="multilingual-e5-large",
        inputs=[emergency_query],
        parameters={"input_type": "passage"}  # Specify the input type
    )[0]['values']  # Get the embedding

    # Query the Pinecone index
    result = index.query(
        vector=query_vector,
        top_k=5,
        include_metadata=True
    )
    return result

# Example of querying the database
emergency_query = "Not getting"
response = query_emergency(emergency_query)

# Print the next step
if response['matches']:
    next_step = response['matches'][0]['metadata']['next_step']
    print(f"Next step: {next_step}")
else:
    print("No matching emergency found.")


Next step: Help the patient sit or lie down in a comfortable position. Offer small sips of clear fluids, like water or ginger ale. Seek medical help if nausea persists or worsens.
